In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

In [2]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

INFO - 2020-11-27 23:56:02,171 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-27 23:56:02,172 - settings - Setting database.user to celiib
INFO - 2020-11-27 23:56:02,173 - settings - Setting database.password to newceliipass


In [24]:
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-28 00:01:24,693 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-28 00:01:24,878 - settings - Setting enable_python_native_blobs to True


# Creating the Exclusion Criteria Table

In [4]:
import datajoint as dj
pinky = dj.create_virtual_module('pinky', 'microns_pinky')
pinky.ExclusionCriteria()

criteria_id key by which to lookup the exclusion criteria description,criteria_desc criteria by which this segment was excluded (preferred to add name of excluder as well)
0,Labeling pipeline
1,postsyn label was error in labeling pipeline
2,All error segments


In [6]:
pinky.ExclusionCriteria.describe()

# Descriptions of exclusion criterias
criteria_id          : int unsigned                 # key by which to lookup the exclusion criteria description
---
criteria_desc        : varchar(64)                  # criteria by which this segment was excluded (preferred to add name of excluder as well)



'# Descriptions of exclusion criterias\ncriteria_id          : int unsigned                 # key by which to lookup the exclusion criteria description\n---\ncriteria_desc        : varchar(64)                  # criteria by which this segment was excluded (preferred to add name of excluder as well)\n'

In [20]:
@schema
class ExclusionCriteria(dj.Manual):
    definition="""
    # Descriptions of exclusion criterias
    criteria_id          : int unsigned                 # key by which to lookup the exclusion criteria description
    ---
    criteria_desc        : varchar(64)                  # criteria by which this segment was excluded (preferred to add name of excluder as well)
    
    """

In [21]:
ExclusionCriteria.insert1(dict(criteria_id=0,criteria_desc="Axon Cleaning 11/27"))

In [22]:
ExclusionCriteria()

criteria_id key by which to lookup the exclusion criteria description,criteria_desc criteria by which this segment was excluded (preferred to add name of excluder as well)
0,Axon Cleaning 11/27


# Creating the Synapse Exclude

In [17]:
pinky.SynapseExclude.describe()

-> pinky.Synapse
-> pinky.ExclusionCriteria



'-> pinky.Synapse\n-> pinky.ExclusionCriteria\n'

In [25]:
@schema
class SynapseExclude(dj.Manual):
    definition="""
    # Descriptions of exclusion criterias
    -> minnie.Synapse
    -> minnie.ExclusionCriteria
    ---
    
    """

In [26]:
SynapseExclude()

synapse_id synapse index within the segmentation,timestamp,criteria_id key by which to lookup the exclusion criteria description


# The labels table that will store all of the error labels

In [30]:
pinky.CompartmentFinal.describe()

-> pinky.PymeshfixDecimatedExcitatoryStitchedMesh



'-> pinky.PymeshfixDecimatedExcitatoryStitchedMesh\n'

In [29]:
minnie.Decomposition()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),decomposition,n_vertices number of vertices,n_faces number of faces,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134026921572,0,0.25,=BLOB=,10752,22449,0,0,0,1,1,1,1,9257.060907158155,9257.060907158155,9257.061,219.966,219.966,219.966,219.966,11,0.0011882821243505153,11.0,9257.061,1,nan,nan,5909217334.438089,9946671.094996769,638347.0297649984,638347.0233628243,5909217334.438089,38.5579
864691134309610836,0,0.25,=BLOB=,14274,31974,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,24.5438
864691134412252598,0,0.25,=BLOB=,13529,29348,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,23.9303
864691134413248927,0,0.25,=BLOB=,14211,31270,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,25.1809
864691134413821752,0,0.25,=BLOB=,11696,25792,1,1,0,1,1,3,3,9299.020064879269,9299.020064879269,1599.272,34.572,34.572,103.183,103.183,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,125.4067
864691134431433407,0,0.25,=BLOB=,11510,24714,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,20.9624
864691134433153501,0,0.25,=BLOB=,11585,23209,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,22.4911
864691134466251864,0,0.25,=BLOB=,10808,23322,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,24.6378
864691134467173584,0,0.25,=BLOB=,14804,31831,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,20.0085
864691134488345090,0,0.25,=BLOB=,12908,27823,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,8.8056


In [32]:
pinky.ProofreadLabel.describe()

-> pinky.PymeshfixDecimatedExcitatoryStitchedMesh
author_original      : varchar(20)                  # name of last editor
author_proofreader   : varchar(20)                  # name of last editor
date_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited
---
vertices             : longblob                     # label data for the vertices
triangles            : longblob                     # label data for the faces
edges                : longblob                     # label data for the edges
status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey



'-> pinky.PymeshfixDecimatedExcitatoryStitchedMesh\nauthor_original      : varchar(20)                  # name of last editor\nauthor_proofreader   : varchar(20)                  # name of last editor\ndate_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited\n---\nvertices             : longblob                     # label data for the vertices\ntriangles            : longblob                     # label data for the faces\nedges                : longblob                     # label data for the edges\nstatus               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey\n'

In [34]:
@schema
class DecompositionErrorLabels(dj.Computed):
    definition="""
    -> minnie.Decomposition
    ---
    n_face_errors : int #the number of faces that were errored out
    face_idx_for_error : longblob #the face indices for the errors computed
    """
    